In [1]:
import utility_common as common
from utility_dfcutter import *
from utility_dfplotter import *
import glob
import pandas as pd

In [3]:
oppoSign  = ' & (lepton1_q != lepton2_q) '
zveto     = ' & (dilepton_mass<80 | dilepton_mass>102) '
lmveto    = ' & (dilepton_mass>12) '
dict_cuts = {
    "mumu"  : "lepton1_pt>25 & lepton2_pt>10 & (nJets>=2) & (nBJets>=1)" + oppoSign + zveto + lmveto,
    "ee"    : "lepton1_pt>30 & lepton2_pt>20 & (nJets>=2) & (nBJets>=1)" + oppoSign + zveto + lmveto,
    "emu"   : "lepton1_pt>25 & (lepton2_pt<30 | (lepton2_pt>30 & lepton2_pt<lepton1_pt))  & (nJets>=2) & (nBJets>=1)" + oppoSign,
    "emu2"  : "lepton2_pt>30 & (lepton1_pt<25 | (lepton1_pt>25 & lepton1_pt<lepton2_pt))  & (nJets>=2) & (nBJets>=1)" + oppoSign,
    "mutau" : "lepton1_pt>30 & lepton2_pt>20 & (nJets>=2) & (nBJets>=1)" + oppoSign + lmveto,
    "etau"  : "lepton1_pt>30 & lepton2_pt>20 & (nJets>=2) & (nBJets>=1)" + oppoSign + lmveto,
    "mu4j"  : "lepton1_pt>30 & (nJets>=4) & (nBJets>=1)",
    "e4j"   : "lepton1_pt>30 & (nJets>=4) & (nBJets>=1)",
}

dict_subtitle = {
    "mumu"  : r"$\mu \mu$ Channel",
    "ee"    : r"$e e$ Channel",
    "emu"   : r"$\mu e$ Channel",
    "emu2"  : r"$e \mu$ Channel",
    "mutau" : r"$\mu \tau$ Channel",
    "etau"  : r"$e \tau$ Channel",
    "mu4j"  : r"$\mu + jets$ Channel",
    "e4j"   : r"$e + jets$ Channel",
}

In [4]:
for selection in dict_subtitle.keys():
    for period in ["2018"]:
        pickleFileDirName = 'emu' if selection == 'emu2' else selection
        pickleDirectry = common.getBaseDirectory() + "data/{}/{}/".format("pickles_"+period,pickleFileDirName)
        subtitle = dict_subtitle[selection]
        cuts = dict_cuts[selection]
        ############################################################################
        # data
        pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("data"))
        data = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
        data.reset_index(drop=True, inplace=True)
        if selection in ["emu","emu2"]:
            data = data.drop_duplicates(subset=['runNumber', 'evtNumber'])
        # tw
        pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mct"))
        mct = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
        mct.reset_index(drop=True, inplace=True)
        # tt
        pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mctt"))
        mctt = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
        mctt.reset_index(drop=True, inplace=True)
        # w
        pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mcw"))
        mcw = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
        mcw.reset_index(drop=True, inplace=True)
        # z
        pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mcz"))
        mcz = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
        mcz.reset_index(drop=True, inplace=True)
        # mcdiboson
        pickles = glob.glob(pickleDirectry + '{}/*.pkl'.format("mcdiboson"))
        mcvv = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
        mcvv.reset_index(drop=True, inplace=True)

        ############################################################################
        dfList   = [mcvv,mcz,mcw,mct,mctt,data]
        legends  = ['VV','Z','W',"t","tt","data"]
        colors   = ["C0", "C1", "C3","C2","C4",'k']
        common.makeDirectory("../plots/test_kinematics_{}/{}/".format(period,selection),clear=False)
        configs = pd.read_csv("../python/plotterItemTables/itemTable_{}.csv".format(pickleFileDirName))
        for index, row in configs.iterrows():
            v,a,b,step,xl = row['var'], row['lower'], row['upper'], row['step'], row['xlabel']
            sk = ASingleKinematicPlot(v,a,b,step,dfList)
            sk.settingPlot(xl,legends,colors,period=period)
            sk.makePlot()
            sk.axes[0].text(0.04*sk.b+0.96*sk.a,1.2*sk.ynorm, subtitle)
            #plt.savefig("../plots/kinematics/{}/{}.pdf".format(selection,v))
            plt.savefig("../plots/test_kinematics_{}/{}/{}.png".format(period,selection,v),dpi=300)
            plt.close()

In [6]:
selection, period = "ee", "2018"

pickleFileDirName = 'emu' if selection == 'emu2' else selection
pickleDirectry = common.getBaseDirectory() + "data/{}/{}/".format("pickles_"+period,pickleFileDirName)
subtitle = dict_subtitle[selection]
cuts = dict_cuts[selection]

for p in "ABCD":
    pickles = glob.glob(pickleDirectry + '{}/*{}.pkl'.format("data",p))
    data = pd.concat([ pd.read_pickle(pickle) for pickle in pickles],ignore_index=True).query(cuts)
    data.reset_index(drop=True, inplace=True)
    print(data.size)

866525
432190
412720
1874345


368.72